In [1]:
import torch
from typing import Callable
import time

def IQR_test(x: torch.tensor, k: int):
    q1, q3 = torch.quantile(x, torch.tensor([0.25, 0.75]))
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    lower_bound = q1 - k * iqr
    if torch.any(x > upper_bound) or torch.any(x < lower_bound):
        return True
    else:
        return False

x = torch.normal(mean=0, std=1, size=(1000, 1000)) * 1e-14
x = x.abs()
x[0][0] = 1e-20
# x[0][0] = 1e+23
# x[0][0] = 2

group_size = 100
min, max = x.view(group_size, -1).aminmax(dim=1)
min = min.where(min != 0, min.max()).log10()

print('fault for min: {}'.format(IQR_test(min, 3)))
print('fault for max: {}'.format(IQR_test(max, 100)))


fault for min: False
fault for max: False


# Testing all theories

In [2]:
def IQR_test(x: torch.tensor, k: int):
    q1, q3 = torch.quantile(x, torch.tensor([0.25, 0.75]).to(x.device))
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    lower_bound = q1 - k * iqr
    if torch.any(x > upper_bound) or torch.any(x < lower_bound):
        return True
    else:
        return False

In [3]:
def vanilla(x: torch.Tensor, q: torch.Tensor = None):
    x = x.abs()
    k = 6
    min, max = torch.aminmax(x)
    mean = torch.mean(x)
    std = torch.std(x)
    return max > mean + k * std

def boost_one_side(x: torch.Tensor, q: torch.Tensor = None):
    x = x.abs()
    max = x.view(group_size, -1).amax(dim=(1,))
    q1, q3 = torch.quantile(max, q)
    return max.max() > q3 + 1_000 * (q3 - q1)

def boost_two_sides(x: torch.Tensor, q: torch.Tensor = None):
    x = x.abs()
    group_size = 100
    # min, max = x.view(group_size, -1).aminmax(dim=1)

    min = x.view(group_size, -1).amin(dim=(1,))
    max = x.view(group_size, -1).amax(dim=(1,))
    min = min.where(min != 0, min.max()).log10()
    return IQR_test(min, 3) or IQR_test(max, 100)

In [4]:
def test(x: torch.Tensor, f: Callable[[torch.Tensor], bool], iters: int, q: torch.Tensor = None):
    start_time = time.time()
    for i in range(iters):
        f(x, q)
    return time.time() - start_time

In [6]:
x = torch.normal(mean=0, std=1e-9, size=(1000, 1000)) * 1e-8
x[0][0] = 1e-30
# x = x.cuda()

result_vanilla = test(x, vanilla, 100)

x = torch.normal(mean=0, std=1e-9, size=(20000, 20000)) * 1e-8
x[0][0] = 1e-30
# x = x.cuda()

result_boost_one_side = test(x, boost_one_side, 100)

x = torch.normal(mean=0, std=1e-9, size=(20000, 20000)) * 1e-8
x[0][0] = 1e-30
# x = x.cuda()

result_boost_two_sides = test(x, boost_two_sides, 100)

print('time for vanilla: {}'.format(result_vanilla))
print('time for boost_one_side: {}'.format(result_boost_one_side))
print('time for boost_two_sides: {}'.format(result_boost_two_sides))

print('optimization for boost_one_side: {}%'.format((result_vanilla / result_boost_one_side - 1) * 100))
print('optimization for boost_two_sides: {}%'.format((result_vanilla / result_boost_two_sides - 1) * 100))

time for vanilla: 0.02778148651123047
time for boost_one_side: 1.1934924125671387
optimization for boost_one_side: -97.67225277524186%


# Results

In [6]:
# CPU results:

# time for vanilla: 45.121389627456665
# time for boost_one_side: 26.617141485214233
# time for boost_two_sides: 35.05351233482361
# optimization for boost_one_side: 69.52004276087085%
# optimization for boost_two_sides: 28.721450782069603%

# CUDA results:
# time for vanilla: 0.0958106517791748
# time for boost_one_side: 0.014071226119995117
# time for boost_two_sides: 1.7375609874725342
# optimization for boost_one_side: 580.897677019265%
# optimization for boost_two_sides: -94.48591143160152%